In [22]:
import numpy as np
from sympy.logic.boolalg import disjuncts

import bench_utils
import csv

def coupling_map_to_pGraph(coupling_map):
    import rustworkx as rx
    MAX_DIST = 1000000
    G = rx.adjacency_matrix(coupling_map.graph)
    C = np.ones((coupling_map.size(), coupling_map.size())) * MAX_DIST
    np.fill_diagonal(C, 0)
    for src, dst in coupling_map.get_edges():
        C[src, dst] = 1
    return G, C

max_dist = 1000000
max_size = 1000000

def is_code_reduced(code):
    if code in ['melbourne', 'manhattan']:
        reduced = True
    else:
        reduced = False
    return reduced

import os


def load_graph(code, dist_comp=False, len_func=lambda x: x):
    # if code == 'sycamore':
    #     return load_sycamore_graph(dist_comp)
    # reduced = is_code_reduced(code)
    
    reduced = False
    
    print('reduced:', reduced)
    pth = os.path.join('../../tetris/arch', 'data', 'ibmq_' + code + '_calibrations.csv')
    cgs = []
    n = 0
    with open(pth, 'r') as cf:
        g = csv.DictReader(cf, delimiter=',', quotechar='\"')
        for i in g:
            cxval = ""
            for j in i.keys():
                if j.find('CNOT') != -1:
                    cxval = i[j]
            n += 1
            if ';' in cxval:
                dc = ';'
            else:
                dc = ','
            for j in cxval.split(dc):
                cgs.append(j.strip())
    if reduced:
        n -= 1  # qubit n is not used
    G = np.zeros((n, n))
    C = np.ones((n, n)) * max_dist
    for i in range(n):
        C[i, i] = 0
    for i in cgs:
        si1 = i.find('_')
        si2 = i.find(':')
        offset = 0
        if i[:2] == 'cx':
            offset = 2
        iq1 = int(i[offset:si1])
        iq2 = int(i[si1 + 1:si2])
        acc = float(i[si2 + 2:]) * 1000
        if (iq1 < n and iq2 < n) or not reduced:
            G[iq1, iq2] = 1
            C[iq1, iq2] = len_func(acc / 1000)
    if dist_comp == True:
        for i in range(n):
            dijkstra(C, i)
    return G, C

def minDistance(dist, sptSet):
    minv = max_size
    min_index = -1
    n = len(dist)
    for v in range(n):
        if dist[v] < minv and sptSet[v] == False:
            minv = dist[v]
            min_index = v
    return min_index

def dijkstra(dist_matrix, src):
    n = dist_matrix.shape[0]
    dist = [dist_matrix[src, i] for i in range(n)]
    sptSet = [False] * n
    for cout in range(n):
        u = minDistance(dist, sptSet)
        if u == -1:
            break
        sptSet[u] = True
        for v in range(n):
            if dist_matrix[u, v] > 0 and sptSet[v] == False and \
                    dist[v] > dist[u] + dist_matrix[u, v]:
                dist[v] = dist[u] + dist_matrix[u, v]
    for i in range(n):
        dist_matrix[src, i] = dist[i]
        dist_matrix[i, src] = dist[i]

In [31]:
G, C = load_graph('manhattan', True)

reduced: False


In [24]:
np.count_nonzero(G)

144

In [32]:
np.unique(C)


array([0.      , 0.008063, 0.008321, ..., 0.283091, 0.287921, 0.296245])

In [33]:
G1, C1 = coupling_map_to_pGraph(bench_utils.Manhattan)

In [34]:
np.count_nonzero(G1)

144

In [36]:
for i in range(65):
    dijkstra(C1, i)
    

array([[0.      , 0.01172 , 0.02698 , ..., 0.267571, 0.287921, 0.280805],
       [0.01172 , 0.      , 0.01526 , ..., 0.257828, 0.278178, 0.269085],
       [0.02698 , 0.01526 , 0.      , ..., 0.242568, 0.262918, 0.253825],
       ...,
       [0.267571, 0.257828, 0.242568, ..., 0.      , 0.02035 , 0.04564 ],
       [0.287921, 0.278178, 0.262918, ..., 0.02035 , 0.      , 0.02529 ],
       [0.280805, 0.269085, 0.253825, ..., 0.04564 , 0.02529 , 0.      ]])

In [35]:
C1.shape

(65, 65)